In [ ]:
#### Animal Detection by Wasim Sayyad ####

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from plyer import notification

In [20]:

IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 10

In [21]:
# these are the paths
train_dir = 'Animal_emotion_dataset/Master Folder/train'
valid_dir = 'Animal_emotion_dataset/Master Folder/valid'
test_dir = 'Animal_emotion_dataset/Master Folder/test'


In [22]:
# augmentation (training images)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [23]:
#validation
valid_test_datagen = ImageDataGenerator(rescale=1./255)


In [24]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Loading validation images
valid_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Loading test images
test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 1216 images belonging to 4 classes.
Found 132 images belonging to 4 classes.
Found 109 images belonging to 4 classes.


In [ ]:
# Main building of model, used max pooling and 2 layers for classification ~W
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])


In [26]:
# Compiling
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [27]:
# I have used a checkpoint to save the best model . 
checkpoint = ModelCheckpoint('animal_emotion_model.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

In [28]:
# Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint]
)

Epoch 1/10
38/38 [==============================] - ETA: 0s - loss: 1.3915 - accuracy: 0.2722
Epoch 1: val_accuracy improved from -inf to 0.33594, saving model to animal_emotion_model.h5
38/38 [==============================] - 42s 1s/step - loss: 1.3915 - accuracy: 0.2722 - val_loss: 1.3829 - val_accuracy: 0.3359
Epoch 2/10


C:\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


38/38 [==============================] - ETA: 0s - loss: 1.3856 - accuracy: 0.2697
Epoch 2: val_accuracy did not improve from 0.33594
38/38 [==============================] - 19s 510ms/step - loss: 1.3856 - accuracy: 0.2697 - val_loss: 1.3783 - val_accuracy: 0.3281
Epoch 3/10
38/38 [==============================] - ETA: 0s - loss: 1.3845 - accuracy: 0.2747
Epoch 3: val_accuracy did not improve from 0.33594
38/38 [==============================] - 19s 509ms/step - loss: 1.3845 - accuracy: 0.2747 - val_loss: 1.3709 - val_accuracy: 0.3281
Epoch 4/10
38/38 [==============================] - ETA: 0s - loss: 1.3821 - accuracy: 0.2763
Epoch 4: val_accuracy did not improve from 0.33594
38/38 [==============================] - 20s 518ms/step - loss: 1.3821 - accuracy: 0.2763 - val_loss: 1.3783 - val_accuracy: 0.3359
Epoch 5/10
38/38 [==============================] - ETA: 0s - loss: 1.3812 - accuracy: 0.2788
Epoch 5: val_accuracy did not improve from 0.33594
38/38 [============================

In [ ]:
# load and prediction
model = load_model('animal_emotion_model.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

class_to_emotion = {0: 'Angry', 1: 'Sad', 2: 'Other', 3: 'Happy'}

filenames = test_generator.filenames

# this is for notifications
for filename, predicted_class in zip(filenames, predicted_classes):
    animal_name = filename.split('/')[0]
    emotion = class_to_emotion[predicted_class]
    # Shorten title and message
    notification_title = f"{animal_name[:15]} - {emotion[:15]} Emotion"
    notification_text = f"{animal_name[:15]} is {emotion[:15]}!"
    notification.notify(title=notification_title, message=notification_text, app_icon=None, timeout=10)

### evaluation (attempt - 4 )###3
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')